In [95]:
import pandas as pd
import numpy as np
import os
import requests
import matplotlib.pyplot as plt
import statsmodels.api as sm
from datetime import datetime
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import log_loss, roc_auc_score, recall_score, precision_score, average_precision_score, f1_score, classification_report, accuracy_score,confusion_matrix, silhouette_score
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.datasets import make_blobs


import warnings
warnings.filterwarnings("ignore")
os.chdir("/Users/yj.noh/Documents/GitHub")
print(os.getcwd())

/Users/yj.noh/Documents/GitHub


## 1. 위경도 - 지번주소 변환

In [ ]:
# 픽업지 지번 주소 추출 

# apiurl = "https://api.vworld.kr/req/address"
# api_key = "57C428EF-A97E-3028-9BF6-734745493260"

# data = pd.read_csv('prj-ML-model-LT_OV30/new_data_pick_address.csv', encoding="utf-8")

# shop_results = []

# for index, row in data.iterrows():
#     lat = row['shop_loc_pnt_lat']
#     lon = row['shop_loc_pnt_lon']
    
#     # 픽업지 주소 추출
#     params_shop = {
#         "service": "address",
#         "request": "getaddress",
#         "crs": "epsg:4326",
#         "point": f"{lon},{lat}",  # 경도, 위도 순서로 입력
#         "format": "json",
#         "type": "parcel",
#         "key": api_key
#     }
    
#     response_shop = requests.get(apiurl, params=params_shop)
    
#     if response_shop.status_code == 200:
#         result_shop = response_shop.json()
#         shop_address = result_shop['response']['result'][0]['text']  # 매장의 지번 주소
            
#     shop_results.append({
#         "Latitude": lat,
#         "Longitude": lon,
#         "Address": shop_address
#         })
    
# shop_result = pd.DataFrame(shop_results)
# shop_result.to_csv('prj-ML-model-LT_OV30/new_data_shop_result.csv', index=False, encoding="cp949")

In [ ]:

# apiurl = "https://api.vworld.kr/req/address"
# api_key = "57C428EF-A97E-3028-9BF6-734745493260"

# data = pd.read_csv('prj-ML-model-LT_OV30/new_data_dlvry_address.csv', encoding="utf-8")

# dlvry_results = []

# for index, row in data.iterrows():
#     lat = row['dlvry_loc_pnt_lat']
#     lon = row['dlvry_loc_pnt_lon']
    
#     # 픽업지 주소 추출
#     params_shop = {
#         "service": "address",
#         "request": "getaddress",
#         "crs": "epsg:4326",
#         "point": f"{lon},{lat}",  # 경도, 위도 순서로 입력
#         "format": "json",
#         "type": "parcel",
#         "key": api_key
#     }
    
#     response_shop = requests.get(apiurl, params=params_shop)
    
#     if response_shop.status_code == 200:
#         result_shop = response_shop.json()
#         shop_address = result_shop['response']['result'][0]['text']  # 매장의 지번 주소
            
#     dlvry_results.append({
#         "Latitude": lat,
#         "Longitude": lon,
#         "Address": shop_address
#         })
    
# dlvry_result = pd.DataFrame(dlvry_results)
# dlvry_result.to_csv('prj-ML-model-LT_OV30/new_data_dlvry_result.csv', index=False, encoding="cp949")

## 2. 지번주소 - 건축물대장 join

In [96]:
# dataload
#픽업지 층 데이터
pick_floor = pd.read_excel("prj-ML-model-LT_OV30/pick_floor_data.xls")

#픽업지/전달지 지번주소 데이터
pick = pd.read_csv("prj-ML-model-LT_OV30/address/new_data_shop_result.csv", encoding = "cp949")
dlvry = pd.read_csv("prj-ML-model-LT_OV30/address/new_data_dlvry_result.csv", encoding = "cp949")

# 건축물대장 데이터
info_1 = pd.read_csv('/Users/yj.noh/Documents/GitHub/prj-ML-model-LT_OV30/address/건축물정보/building_final.csv', encoding = "cp949")
info_2 = pd.read_csv('/Users/yj.noh/Documents/GitHub/prj-ML-model-LT_OV30/address/건축물대장_all.csv', encoding = "cp949")


In [97]:
print(info_1.head(2)) # address로 pick, dlvry join

             address  건물용도명  지상층수_max  지하층수_min  n  pick  dlvry
0  서울특별시 서초구 방배동 1-3  일반음식점       6.0         2  1     1      1
1  서울특별시 서초구 방배동 1-4   공동주택       2.0         1  1     1      1


In [98]:
print(pick.isna().sum())

Latitude     0
Longitude    0
Address      0
dtype: int64


In [99]:
info_1_pick = info_1[info_1['pick'] == 1]
info_1_dlvry = info_1[info_1['dlvry'] == 1]

print(info_1_pick.columns)
print(pick.columns)
print(pick.shape) #1,336

Index(['address', '건물용도명', '지상층수_max', '지하층수_min', 'n', 'pick', 'dlvry'], dtype='object')
Index(['Latitude', 'Longitude', 'Address'], dtype='object')
(1336, 3)


## 2-1. info_1 join

In [100]:
pick = pd.merge(pick, info_1_pick[['address', '건물용도명']], 
                left_on = ['Address'], right_on = ['address'], how = 'left')

print(pick.isna().sum()) #183개
print(pick.shape)

Latitude       0
Longitude      0
Address        0
address      183
건물용도명        183
dtype: int64
(1336, 5)


In [101]:
dlvry = pd.merge(dlvry, info_1_dlvry[['address', '건물용도명']], 
                left_on = ['Address'], right_on = ['address'], how = 'left')

print(dlvry.isna().sum()) # 5,125개
print(dlvry.shape)

Latitude        0
Longitude       0
Address         0
address      5125
건물용도명        5125
dtype: int64
(17824, 5)


## 2-2. info_2 join

In [104]:
info_2 = info_2[["주용도코드명", "대지위치"]]
pick = pd.merge(pick, info_2, 
                left_on="Address", right_on="대지위치", how="left")

dlvry = pd.merge(dlvry, info_2, left_on="Address", right_on="대지위치", how="left")


pick["건물용도명_2"] = pick.apply(lambda row: row["주용도코드명"] if pd.isna(row["건물용도명"]) else row["건물용도명"], axis=1)
dlvry["건물용도명_2"] = dlvry.apply(lambda row: row["주용도코드명"] if pd.isna(row["건물용도명"]) else row["건물용도명"], axis=1)


In [105]:
print(pick.isna().sum()) 
print(dlvry.isna().sum())

Latitude        0
Longitude       0
Address         0
address       183
건물용도명         183
주용도코드명       1292
대지위치         1290
건물용도명_2       169
dtype: int64
Latitude         0
Longitude        0
Address          0
address       5131
건물용도명         5131
주용도코드명       13793
대지위치         13766
건물용도명_2       3823
dtype: int64
